# Document (QnA) Matching Data Science Process

## Part 3: TF-IDF and Cosine Similarity - Match Q to Q, then Link to A

### Overview

This notebook is Part 3 of 5, in a series providing a step-by-step description of how to create discriminative training methods to match the correct answer to a given question. Using Python packages and custom code examples, we have implemented the basic framework that combines key phrase learning and latent topic modeling as described in the paper entitled ["Modeling Multiword Phrases with Constrained Phrases Tree for Improved Topic Modeling of Conversational Speech"](http://people.csail.mit.edu/hazen/publications/Hazen-SLT-2012.pdf) which was originally presented in the 2012 IEEE Workshop on Spoken Language Technology. 

Although the paper examines the use of the technology for analyzing human-to-human conversations, the techniques are quite general and can be applied to a wide range of natural language data including news stories, legal documents, research publications, social media forum discussions, customer feedback forms, product reviews, and many more.

Also, we implement a Naive Bayes Classifier as described in the paper entitled ["MCE Training Techniques for Topic Identification of Spoken Audio Documents"](http://ieeexplore.ieee.org/abstract/document/5742980/).

Part 3 of the series shows the process of matching a test question to a similar training question based on the Cosine Similarities of their TF-IDF vectors. We believe the answer that resolved the training question could also resolve the test question.

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/experiment2_overview.PNG?token=APoO9vukygoptytwwbh67JZ1YSFKG7COks5Ynhv0wA%3D%3D">

Note: This notebook series are built under Python 3.5 and NLTK 3.2.2.

## Import required Python modules

In [1]:
import pandas as pd
import math
import numpy as np
from numpy import linalg as LA
from azure.storage import CloudStorageAccount
from IPython.display import display

# suppress all warnings
import warnings
warnings.filterwarnings("ignore")

## Read trainQ and testQ into DataFrames

In [3]:
trainQ_url = 'https://mezsa.blob.core.windows.net/stackoverflow/trainQwithTokens.tsv'
testQ_url = 'https://mezsa.blob.core.windows.net/stackoverflow/testQwithTokens.tsv'

trainQ = pd.read_csv(trainQ_url, sep='\t', index_col='Id', encoding='latin1')
testQ = pd.read_csv(testQ_url, sep='\t', index_col='Id', encoding='latin1')

## Create Tokens to IDs Hash

For each token in the entire vocabulary, we assign it an unique ID.

In [4]:
# get Token to ID mapping: {Token: tokenId}
def tokens_to_ids(tokens, featureHash):
    token2IdHash = {}
    for i in range(len(tokens)):
        tokenList = tokens.iloc[i].split(',')
        if featureHash is None:
            for t in tokenList:
                if t not in token2IdHash.keys():
                    token2IdHash[t] = len(token2IdHash)
        else:
            for t in tokenList:
                if t not in token2IdHash.keys() and t in list(featureHash.keys()):
                    token2IdHash[t] = len(token2IdHash)
            
    return token2IdHash

In [5]:
token2IdHashInit = tokens_to_ids(trainQ['Tokens'], None)

In [7]:
print("Total number of unique tokens in the TrainQ: " + str(len(token2IdHashInit)))

Total number of unique tokens in the TrainQ: 4977


## Create Count Matrix for Each Token in Each Question

In [8]:
def count_matrix(frame, token2IdHash, uniqueAnswerId):
    # create am empty matrix with the shape of:
    # num_row = num of unique tokens
    # num_column = num of unique answerIds (N_wA) or num of questions
    # rowIdx = token2IdHash.values()
    # colIdx = index of uniqueAnswerId (N_wA) or index of questions
    num_row = len(token2IdHash)
    if uniqueAnswerId is not None:  # get N_wA
        num_column = len(uniqueAnswerId)
    else: 
        num_column = len(frame)
    countMatrix = np.empty(shape=(num_row, num_column))

    # loop through each question in the frame to fill in the countMatrix with corresponding counts
    for i in range(len(frame)):
        tokens = frame['Tokens'].iloc[i].split(',')
        if uniqueAnswerId is not None:   # get N_wA
            answerId = frame['AnswerId'].iloc[i]
            colIdx = uniqueAnswerId.index(answerId)
        else:
            colIdx = i
            
        for t in tokens:
            if t in token2IdHash.keys():
                rowIdx = token2IdHash[t]
                countMatrix[rowIdx, colIdx] += 1

    return countMatrix

In [9]:
# calculate the count matrix of all training questions.
N_wQ = count_matrix(trainQ, token2IdHashInit, None)

## Compute IDF Vector

Considering all tokens observed in the training questions and answers, we compute their Inverse Document Frequency based on the below formula.

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/idf.PNG?token=APoO9qf3WptQgUPVRQJuOt4cobf56-Y3ks5YnhLSwA%3D%3D">

In [10]:
def get_idf(N_wQ):
    # N is total number of documents in the corpus
    # N_V is the number of tokens in the vocabulary
    N_V, N = N_wQ.shape
    # D is the number of documents where the token w appears
    D = np.empty(shape=(0, N_V))
    for i in range(N_V):
        D = np.append(D, len(np.nonzero(N_wQ[i, ])[0]))
    return np.log(N/D)

In [11]:
idf = get_idf(N_wQ)

## Calculate Normalized TF of Each Word w in Training and Test Sets

Each document d is typically represented by a feature vector x that represents the contents of d. Because different documents can have different lengths, it can be useful to apply L1 normalmalized feature vector x. Therefore, a normalized Term Frequency matrix can be obtained based on the below formula.

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/tf.PNG?token=APoO9tMyEVzqoUJYT9ALcdF3_BryHHEVks5YnIQywA%3D%3D">

In [12]:
def normalize_tf(frame, token2IdHash):
    N_w = count_matrix(frame, token2IdHash, None)
    # get the column sum of the count matrix
    N_W = np.sum(N_w, axis=0)                                                                                              
    
    # find the index where N_W is zero and remove those columns from N_w
    removeIdx = np.where(N_W == 0)[0]
    
    if len(removeIdx) > 0:
        # remove the questions where the TF is NaN
        N_w = np.delete(N_w, removeIdx, axis=1)
        N_W = np.delete(N_W, removeIdx, axis=0)
        frame = frame.drop(frame.index[removeIdx])
        
    # x_w = P_wd = count(w)/sum(count(i in V))
    x_w = N_w / N_W
    
    return x_w, frame

In [13]:
# calculate tf matrix on trainQ and testQ independently.
x_wTest, testQ = normalize_tf(testQ, token2IdHashInit)
x_wTrain = normalize_tf(trainQ, token2IdHashInit)[0]

## Compute TF-IDF Matrix

By knowing the Term Frequency (TF) matrix and Inverse Document Frequency (IDF) vector, we can simply compute TF-IDF matrix by multiplying them together.

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/tfidf.PNG?token=APoO9gw3rPhLusbG3if65TuVZNAnyqTCks5YnhWPwA%3D%3D">

In [14]:
tfidfTest = (x_wTest.T * idf).T
tfidfTrain = (x_wTrain.T * idf).T

## Calculate Cosine Similarity between Training and Test Sets

For each question in the Test set, we compute its Consine Similarity against all questions in the Training set. This similarity is a score that we use to consider how similar a question and an answer is. 

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/cosine.PNG?token=APoO9lOcKkP_7tFDh7p6KZXXVmokwLbGks5YnhY-wA%3D%3D">

In [15]:
def consine_similarity(tfidfL, tfidfR):
    # calculate the dot product of two tfidf arrays
    N = np.dot(tfidfL.T, tfidfR)
    # calculate the norm of each tfidf array
    normL = LA.norm(tfidfL, axis = 0)
    normR = LA.norm(tfidfR, axis = 0)
    similarity = (N.T/normL).T/normR
    
    return similarity

In [16]:
# calculate similarity scores of each question in Test set against all training questions. 
simScores = consine_similarity(tfidfTest, tfidfTrain)

## Rank the Cosine Similarity and Calcualte Average Rank 

We use two evaluation matrices to test our model performance. For each question in the test set, we calculate a Cosine Similarity score against each question in the training set. For that test question, we then calculate the average Cosine Similarities per AnswerId. Based on the average similarities, we rank the answers to calculate Average Rank and Top 10 Percentage in the Test set using the below formula:

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/evaluation.PNG?token=APoO9hyYDFxGc9FRbmIXU3VGv0wdeCaPks5YnIVtwA%3D%3D">

In [70]:
# for each question in test set, calculate the average cosine similarities of training questions per answer group
def average_similarity(frame, simScores, uniqueAnswerId):
    # create an empty matrix to store the average similarities per answer group
    # shape = (num_testQ, num_uniqueAnswerId)
    avgSimScores = np.empty(shape = (simScores.shape[0], len(uniqueAnswerId)))

    # get indices of all answerIds
    answerIdxs = []

    for i in np.array(frame['AnswerId']):
        if i not in uniqueAnswerId:
            continue
        else:
            answerIdxs.append(uniqueAnswerId.index(i))
    
    # for each question in test set, calculate the average similarity per answer group
    for j in range(len(avgSimScores)):
        avgSimScores[j, ] = np.bincount(answerIdxs, weights = simScores[j, ]) / np.bincount(answerIdxs)
        
    return avgSimScores

In [71]:
# sort the similarity scores in descending order and map them to the corresponding AnswerId in Answer set
def rank(frame, scores, uniqueAnswerId):
    frame['SortedAnswers'] = list(np.array(uniqueAnswerId)[np.argsort(-scores, axis=1)])
    
    rankList = []
    for i in range(len(frame)):
        rankList.append(np.where(frame['SortedAnswers'].iloc[i] == frame['AnswerId'].iloc[i])[0][0] + 1)
    frame['Rank'] = rankList
    
    return frame

In [72]:
# get unique answerId in ascending order.
uniqueAnswerId = list(np.unique(trainQ['AnswerId']))
# get average cosine similarity.
avgSimScores = average_similarity(trainQ, simScores, uniqueAnswerId)
# calculate the rank of each question in Test set.
testQ = rank(testQ, avgSimScores, uniqueAnswerId)

In [73]:
# average of rank
print('Average of rank: ' + str(np.floor(testQ['Rank'].mean())))
print('Total number of questions in test set: ' + str(len(testQ)))
print('Total number of answers: ' + str(len(uniqueAnswerId)))
print('Total number of unique features: ' + str(len(token2IdHashInit)))
print('Percentage of questions find answers in top 10: ' + str(round(len(testQ.query('Rank <= 10'))/len(testQ), 3)))

Average of rank: 70.0
Total number of questions in test set: 3671
Total number of answers: 1275
Total number of unique features: 4977
Percentage of questions find answers in top 10: 0.385


In [75]:
# check some results.
# uncomment it to execute.
# testQ.query('Rank <= 3')

## Results and Conclusion of Part 2 and 3

As we have experimented two approaches in Part 2 and 3, we notice that using the answer of a similar question to resolve a new question is a better approach (Part 3). There are substantially more words in common among questions than that between questions and answers. 

We also experiment a different phrases learning apporach by leveraging the [phrase detection functions in Gensim package](https://radimrehurek.com/gensim/models/phrases.html). The results are less favorable than our custom approach described in Part 1.

Here is a comparison matrix of two phrases learning approaches and two experiments (part 2 and part 3).

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/tfidf_results.PNG?token=APoO9phH-e-G9nS_23ej8qjyXP1V6cAdks5Yni46wA%3D%3D">